#### Задача:

При тестировании гипотезы целевой группе предложена новая механика оплаты услуг на сайте, у контрольной группы остается базовая механика. Проанализировать результаты АБ-теста и сделать вывод о том, стоит ли запускать новую механику оплаты на всех пользователей.

Данные:

groups - информация о принадлежности пользователя к контрольной группе или тестовой группе (А- контроль Б - тест);
add - файл.который присладаи по прошествии 2 дней после передачи данных;
active - данные пользователей, которые зашли на платформу в период проведения эксперимента;
checks - информация о платежах пользователей.

#### Алгоритм решения:

Импортирую библиотеки.
Загружаю датасеты.
Смотрю структуру, типы данных, проверяю на наличие дубликатов и пропущенных значений.
Формирую рабочий датафрейм из загруженных датасетов.
Провожу разделение датасета на тестовую и контрольную группы.
Выделяю основные метрики для анализа для принятия решения об эффективности изменений по результатам АБ-теста.
Проверяю насколько изменились основные параметры в тестовой группе относительно контрольной группы.
Строю боксплоты для того чтобы оценить выбросы.
Оцениваю наличие и величину выбросов и принимаю решение что с ними делать.
Проверяю тестовую и контрольную выборки на нормальность.
Проверяю равномерность распределения данных в контрольной и тестовой группах.
Выбираю методы проверки статистической значимости различий выбранных метрик в тестовой и контрольной группах.
Осуществляю проверку на статистическую значимость. В зависимости от наличия выбросов с ними и без.
Делаю финальные выводы.

#### Импортирую библиотеки:

In [76]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm.auto import tqdm
from scipy.stats import norm, mannwhitneyu, ttest_ind, chi2_contingency, chi2, levene, shapiro

#### Загружаю информацию:

In [46]:
students = pd.read_csv('./datasets/2/active.csv', sep=';')

groups = pd.read_csv('./datasets/2/groups.csv', sep=';')

add = pd.read_csv('./datasets/2/add.csv',sep=',')

checks = pd.read_csv('./datasets/2/checks.csv', sep=';')

#### Смотрю на основные параметры датасетов:

In [72]:
def dup_na(df):
    
    return print(f'В этом датасете {df.isna().sum()} пропущенных значений и {df.duplicated().sum()} дубликатов. Его размер {df.shape}.')

In [71]:
dup_na(students)

В этом датасете 0 пропущенных значений и 0 дубликатов. Его размер (8341, 1).


In [73]:
dup_na(groups)

В этом датасете id     0
grp    0
dtype: int64 пропущенных значений и 0 дубликатов. Его размер (74484, 2).


In [74]:
dup_na(add)

В этом датасете id     0
grp    0
dtype: int64 пропущенных значений и 0 дубликатов. Его размер (92, 2).


In [75]:
dup_na(checks)

В этом датасете student_id    0
rev           0
dtype: int64 пропущенных значений и 0 дубликатов. Его размер (541, 2).


#### Посмотрю на вид датасетов:

In [6]:
students.head(2)

,student_id
0,581585
1,5723133


In [7]:
groups.head(2)

,id,grp
0,1489,B
1,1627,A


In [11]:
add.head(2)

,id,grp
0,5694584,B
1,5694830,B


In [12]:
checks.head(2)

,student_id,rev
0,1627,990.0
1,3185,690.0


#### Формирую общий датафрейм:

In [82]:
df = students.merge(checks, how='left', on='student_id')

groups = groups.rename(columns={'id': 'student_id'})

df = df.merge(groups, how='left', on='student_id')

df.head()

,student_id,rev,grp
0,581585,NaN,A
1,5723133,NaN,NaN
2,3276743,NaN,B
3,4238589,NaN,A
4,4475369,NaN,B


#### Интересно, сколько посетителей сайта не принадлежат ни к оной группе:

In [31]:
df.grp.isnull().sum()

13

In [40]:
df.isna().sum()

student_id       0
rev           7949
grp             13
dtype: int64

In [87]:
# Уберу этих посетителей из датасета:

df = df.dropna(subset='grp')

df = df.fillna(0)

####  Проверю насколько равномерно распределены данные межу группами - для этого сравню конверсии:

In [137]:
users = df.groupby('grp', as_index=False) \
          .agg({'rev': 'count'}) \
          .rename(columns={'rev': 'num_users'})

users['num_clients'] = df.query('rev > 0') \
                         .groupby('grp', as_index=False) \
                         .agg({'rev': 'count'}) \
                         .rev

users['cr']= users.num_clients / users.num_users * 100

users

,grp,num_users,num_clients,cr
0,A,1535,78,5.081433
1,B,6793,313,4.607684


#### Исходя из конверсий в группах А и Б делаю вывод о том что данные распределены достаточно равномерно.

#### В качестве метрик выберу:

Средний чек платящих пользователей;

конверсию из активных пользователей в клиентов:

CR = количество клиентов / количеству активных пользователей

#### Выделю тестовую иконтрольную группы:

In [91]:
control = df.query('grp == "A"')

test = df.query('grp == "B"')